In [1]:
import json 
import pandas as pd
import numpy as np
import time
from scipy.sparse import dok_matrix
import itertools
from sklearn import preprocessing
import os

In [33]:
# importar varios json 
nombres = [i for i in os.listdir('./mpd.v1/data/') if i.endswith('.json')]

data_various = []
for i in nombres:
    data_various.append(json.load(open('./mpd.v1/data/{}'.format(i))))

In [34]:
#elegir cuantos json (c/u tiene 1000 listas)
n_json = 10

In [35]:
# listas de listas de canciones 
listas = {}
cont = 0 

song_index = {}
index_song = {}
cont_song = 0

# cambiar 'l' a "data" y quitar esta iteracion si queremos hacerlo con 1 json  
for l in data_various[:n_json]:
    for i in range(1000):

        listas[l['playlists'][i]['pid']] = [x['track_name'] for x in l['playlists'][i]['tracks']]
        for x in listas[l['playlists'][i]['pid']]:
            if not x in song_index:
                song_index[x] = cont_song
                index_song[cont_song] = x
                cont_song += 1


In [36]:
len(song_index)

132920

In [ ]:
inicio = time.time()
size = len(song_index)

matriz = dok_matrix((size,size),dtype='uint')

for lista in listas:
    for (i,j) in itertools.combinations(listas[lista],2):
        if i!=j:
            matriz[song_index[i],song_index[j]] += 1
               
fin = time.time()

print('tiempo demorado: {} segs'.format(fin - inicio))

In [ ]:
# threshold del weight
threshold = 10

In [ ]:
now = time.time()
pairs = []
in_pair = set()
for x,y in matriz.keys():
    if matriz[(x,y)] > threshold:
        pairs.append([x,y,matriz[(x,y)]])
        in_pair.add(x)
        in_pair.add(y)

print('tiempo demorado: {} segs'.format(time.time() - now))

In [ ]:
pd.DataFrame(pairs, columns = ['Source','Target', 'Weight']).to_csv('spotify_graph.csv', sep=';', index= False)

In [ ]:
pairs_2 = [[x,index_song[x]] for x in in_pair]

In [ ]:
pd.DataFrame(pairs_2, columns = ['id', 'label']).to_csv('spotify_labels.csv', sep=';', index=False)